In [1]:
# Importing the required packages
import os
import json
import rasterio
import rasterio.features
import rasterio.mask
import shapely.geometry
import pandas as pd
from affine import Affine
import numpy as np
from matplotlib import pyplot as plt
import csv

In [2]:
'''
Raster function for india to get total luminosity of India  

Open the source image as a raster object, to get India we crop raster by calling rasterio.mask.mask function 
documentation of rasterio.mask.mask:' https://rasterio.readthedocs.io/en/latest/api/rasterio.mask.html#rasterio.mask.raster_geometry_mask'

Input: The function has parameters first is raster object second is geometry of region of interest here it is India's geometry, which is extracted from json file of India,

Output: The outputs of mask function have a tuple which has intensity of India, we sum all elements to get total intensity
'''
    
def raster_array_India(image):
    sumi=0
    data=json.load(open("json/india.json"))
    ind_geom=[data['geometry']]

    with rasterio.open(image) as src:
        out_imageind,out_transform = rasterio.mask.mask(src, ind_geom, all_touched=True, nodata=0.0,crop=True)
    raster_array=out_imageind[0]   
    for row in range (len(raster_array)):
         for col in range(len(raster_array[0])):
                if(raster_array[row][col]<0.0):
                    raster_array[row][col]=0.0
                sumi = sumi + raster_array[row][col]
    return sumi


'''
Raster array function for states, to get luminosity of a state
1. We open the source image as a raster object, to get the reqired state we crop raster by calling rasterio.mask.mask function 
documentation of rasterio.mask.mask:' https://rasterio.readthedocs.io/en/latest/api/rasterio.mask.html#rasterio.mask.raster_geometry_mask'
2. Input : The function has parameters first is raster object second is geometry of region of interest here it is the state's geometry, which is extracted from json file of India's states ,
3. Output : The outputs of mask function have a tuple which has intensity of the state, we sum all elements to get total intensity  
'''

def raster_array_state(image,state):
    sumi=0
    for x in data_states['features']:
        if (x['id']==state):
            state_geo=[x['geometry']]
    
    with rasterio.open(image) as src:
        out_imagest,out_transform = rasterio.mask.mask(src,state_geo , all_touched=True, nodata=0.0,crop=True)   
    raster_array=  out_imagest[0]
    for row in range (len(raster_array)):
            for col in range(len(raster_array[0])):
                if(raster_array[row][col]<0.0):
                    raster_array[row][col]=0.0
                sumi = sumi + raster_array[row][col]
    return sumi

In [3]:
# Store all the VIIRS tif files in "files"
import os
path = '../dataset/dmsp-ols/'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.tif' in file:
            files.append(os.path.join(r, file))

# # Print the filename for verification
for f in files:
    print(f)

../dataset/dmsp-ols/F101992.v4b.avg_lights_x_pct/F101992.v4b.pct_lights.tif
../dataset/dmsp-ols/F101992.v4b.avg_lights_x_pct/F101992.v4b.avg_lights_x_pct.tif
../dataset/dmsp-ols/F101993.v4b.avg_lights_x_pct/F101993.v4b.pct_lights.tif
../dataset/dmsp-ols/F101993.v4b.avg_lights_x_pct/F101993.v4b.avg_lights_x_pct.tif
../dataset/dmsp-ols/F101994.v4b.avg_lights_x_pct/F101994.v4b.pct_lights.tif
../dataset/dmsp-ols/F101994.v4b.avg_lights_x_pct/F101994.v4b.avg_lights_x_pct.tif
../dataset/dmsp-ols/F121994.v4b.avg_lights_x_pct/F121994.v4b.pct_lights.tif
../dataset/dmsp-ols/F121994.v4b.avg_lights_x_pct/F121994.v4b.avg_lights_x_pct.tif
../dataset/dmsp-ols/F121995.v4b.avg_lights_x_pct/F121995.v4b.pct_lights.tif
../dataset/dmsp-ols/F121995.v4b.avg_lights_x_pct/F121995.v4b.avg_lights_x_pct.tif
../dataset/dmsp-ols/F121996.v4b.avg_lights_x_pct/F121996.v4b.pct_lights.tif
../dataset/dmsp-ols/F121996.v4b.avg_lights_x_pct/F121996.v4b.avg_lights_x_pct.tif
../dataset/dmsp-ols/F121997.v4b.avg_lights_x_pct/F12

In [4]:
# Summing for India and saving in Indiadata.csv
with open('Indiadmsp.csv','w', newline='') as f:
    thewriter=csv.writer(f)
    for vf in files:
        print(vf[49:56] + vf[72:73] + str(raster_array_India(vf)))
        thewriter.writerow([vf[27:33],vf[72:73],raster_array_India(vf)])

KeyboardInterrupt: 

In [ ]:
''' Makes list a of staes in json file to give as input in raster_array_state function '''

states=[]
data_states=json.load(open("json/states.json"))
for x in data_states["features"]:
    states.append(x['id'])

In [ ]:
# Summing for India and saving in Indiadata.csv
with open('IndiaStatedmsp.csv','w', newline='') as f:
    thewriter=csv.writer(f)
    for vf in files:
        for i in states:
            arr = raster_array_state(vf,i)
            print(str(i) + str(arr))
            thewriter.writerow([vf[27:33],vf[72:73],i,arr])